In [1]:
begin
    using Pkg
    Pkgs = ["CSV", "DataFrames", "Ipopt", "JuMP", "Plots", "PowerModels", 
            "PowerPlots", "DelimitedFiles", "LaTeXStrings", "SparseArrays", "NLopt"]
    for p in Pkgs
        if Base.find_package(p) == nothing
            Pkg.add(p)
        end
    end

    using CSV, DataFrames, Ipopt, JuMP, Plots, PowerModels, PowerPlots, DelimitedFiles, LaTeXStrings, SparseArrays, NLopt
end

In [2]:
S_base = readdlm("data/BaseMVA.txt")[1,1]

10.0

In [3]:
BusData = CSV.read("data/bus.csv", DataFrame)
GenData = CSV.read("data/gen.csv", DataFrame)
GenCostData = CSV.read("data/gencost.csv", DataFrame)
BranchData = CSV.read("data/branch.csv", DataFrame)
DemandData = CSV.read("data/demand.csv", DataFrame)

,1,2,3,4,5,6,7,8,9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.2195,19.075,18.312,17.549,17.549,17.549,20.601,22.89,26.235


In [4]:
GenCostData

,2,startup,shutdown,n,c2,c1,c0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.0,1500.0,0.0,3.0,0.0,40.0,0.0
2,2.0,2000.0,0.0,3.0,0.0,160.0,0.0


In [5]:
BusData

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [6]:
NBuses = size(BusData)[1]
NLines = size(BranchData)[1]
Y = zeros(Complex, NBuses, NBuses)
b = zeros(NBuses, NBuses)
X = zeros(NBuses, NBuses)
BranchData[!,"fbus"] = convert.(Int64, BranchData[!,"fbus"])
BranchData[!,"tbus"] = convert.(Int64, BranchData[!,"tbus"])

for row in eachrow(BranchData)
    # Non-diagonal elements of Y
    Y[row["fbus"], row["tbus"]] = Y[row["fbus"], row["tbus"]] - (1 / (row["r"] + im*row["x"])) 
    Y[row["tbus"], row["fbus"]] = Y[row["tbus"], row["fbus"]] - (1 / (row["r"] + im*row["x"]))
    
    # Diagonal elements of Y
    Y[row["fbus"], row["fbus"]] = Y[row["fbus"], row["fbus"]] + (1 / (row["r"] + im*row["x"])) 
    Y[row["tbus"], row["tbus"]] = Y[row["tbus"], row["tbus"]] + (1 / (row["r"] + im*row["x"]))
    
    # Shunt susceptance
    b[row["fbus"], row["tbus"]] = row["b"]
    b[row["tbus"], row["fbus"]] = row["b"]
    
    # Reactence matrix
    X[row["fbus"], row["tbus"]] = row["x"]
    X[row["tbus"], row["fbus"]] = row["x"]
end

G = real(Y)
B = imag(Y)

Lines = [(row["fbus"], row["tbus"]) for row in eachrow(BranchData)]

1-element Vector{Tuple{Int64, Int64}}:
 (1, 2)

In [7]:
Y

2×2 Matrix{Complex}:
 0.0-10.0im  0.0+10.0im
 0.0+10.0im  0.0-10.0im

In [8]:
# Lines' apparent power limits

S = zeros(NBuses,NBuses)
for i in 1:NLines
    S[BranchData[i,:fbus],BranchData[i,:tbus]] = BranchData[i,:rateA] # Choose rateA column because it is for the long-term case
    S[BranchData[i,:tbus],BranchData[i,:fbus]] = BranchData[i,:rateA]
end

In [9]:
Gens = 1:size(GenData)[1]
Buses = 1:size(BusData)[1]
T = 1:24

1:24

In [10]:
function get_data_OPF(m)
    
    """
    This function parses data from a JuMP model and constructs 3 DataFrames: Branch Data, Bus Data and Gen Data
    and returns objective value of the model
    
    Arguments:
        m (JuMP model) : function gets data from this model

    Returns:
        BranchData (DataFrame) : DataFrame that contains information about power flows in every branch 
        BusData    (DataFrame) : DataFrame that contains information about voltages magnitudes and angles in every bus
        GenData    (DataFrame) : DataFrame that contains information about power generated in every generator
        J          (Float64)   : Value of objective function
    """
    
    # Obtain Branch Data
    BranchData = DataFrame(from_bus = Int64[], to_bus = Int64[], 
                            reactive_power_to = Float64[], reactive_power_from = Float64[],
                            active_power_to = Float64[], active_power_from = Float64[])
    
    pijs = value.(m[Symbol("pij")])
    qijs = value.(m[Symbol("qij")])
    
    for line in Lines
        push!(BranchData, (line[1], line[2], qijs[line[1], line[2]], qijs[line[2], line[1]], 
                                             pijs[line[1], line[2]], pijs[line[2], line[1]]))
    end
    
    # Obtain Bus Data
    BusData = DataFrame(Vm = Float64[], Va = Float64[])
    v = value.(m[Symbol("v")])
    δ = value.(m[Symbol("δ")])
    
    for i in eachindex(v)
        push!(BusData, (v[i][1], δ[i][1]))
    end
    
    # Obtain Gen Data
    GenData = DataFrame(Pg = Float64[], Qg = Float64[])
    p = value.(m[Symbol("p")])
    q = value.(m[Symbol("q")])
    
    for i in eachindex(p)
        push!(GenData, (p[i], q[i]))
    end
    
    # Obtain objective function value
    J = objective_value(m)
    
    return BranchData, BusData, GenData, J
end 

get_data_OPF (generic function with 1 method)

In [11]:
# Тут функция с OPF, без линеаризации
function OPF(G, B, b, S, BusData, DemandData, GenData, Gens, Buses, T)
    """
    This function creates a JuMP model for Optimal Power Flow (OPF) problem with provided parameters of power grid
    
    Arguments:
        G (Matrix{Real})        : conductance matrix of grid -- real part of admittance matrix
        B (Matrix{Real})        : susceptance matrix of grid -- imaginary part of admittance matrix
        b (Matrix{Float64})     : shunt matrix for lines
        S (Matrix{Float64})     : line flow limits matrix
        BusData (DataFrame)     : contains information on buses: power demands and voltage magnitude limits
        GenData (DataFrame)     : contains information on generators: generation limits and quadratic cost function's coefficients
        Gens (UnitRange{Int64}) : indices of generators
        Buses (UnitRange{Int64}): indices of buses
    Returns:
        
    """
    
    m = Model(Ipopt.Optimizer)
    Emax = 100.0
    Emin = 10.0
    η_ch = 1.0 #0.8
    η_disch = 0.85
    # Variables
    ## Voltages
    @variable(m, BusData[i, :Vmin] ≤ v[i in Buses, t in T] ≤ BusData[i, :Vmax], start = 1.0)
    @variable(m, δ[i in Buses, t in T], start = 0.0)
    
    ## Power generations
    @variable(m, GenData[i, :Pmin] ≤ p[i in Gens, t in T] ≤ GenData[i, :Pmax])
    @variable(m, GenData[i, :Qmin] ≤ q[i in Gens, t in T] ≤ GenData[i, :Qmax])
        
    # Battery
    @variable(m, Emin ≤ E[t in T] ≤ Emax)
    @variable(m, 0.0 ≤ Pch[i in Buses, t in T] ≤ 25.0)
    @variable(m, 0.0 ≤ Pdisch[i in Buses, t in T] ≤ 40.0)
    
    # No simultaneous charge and discharge
    @constraint(m, Psim[i in Buses, t in T], Pch[i, t] * Pdisch[i, t] == 0.0)
    
    ## Battery energy
    @constraint(m, Et1, E[1] == Emin)
    @constraint(m, Pch2, Pch[2, 1] == 0.0)
    @constraint(m, Pdisch2, Pdisch[2, 1] == 0.0)
    @constraint(m, Pch1[t in T], Pch[1, t] == 0.0)
    @constraint(m, Pdisch1[t in T], Pdisch[1, t] == 0.0) # / η_disch  
    @constraint(m, Et[t in 2:24], E[t] == E[t - 1] - (Pdisch[2, t - 1]) +  (η_ch * Pch[2, t - 1]))
    @constraint(m, Pdisch21[t in T], E[t] - Pdisch[2, t] ≥ Emin )
    @constraint(m, Pch21[t in T], E[t] + η_ch * Pch[2, t] ≤ Emax )
    
    ## Power flows
    @variable(m, pij[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)])
    @variable(m, qij[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)])
    
    #Constraints
    
    ##Slack bus
    @constraint(m, δ_slack[t in T], δ[1, t] == 0.0)
    
    ## Power flow through lines
    @NLconstraint(m, p_line[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                     pij[i, j, t] == S_base * (v[i, t] * v[j, t] *(G[i, j] * cos(δ[i, t] - δ[j, t]) +
                                                           B[i, j] * sin(δ[i, t] - δ[j, t])) - G[i,j] * v[i, t]^2) 
    )
    @NLconstraint(m, q_line[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                     qij[i, j, t] == S_base * (v[i, t] * v[j, t] * (G[i, j] * sin(δ[i, t] - δ[j, t]) -
                                                           B[i, j] * cos(δ[i, t] - δ[j, t])) + 
                                                          (B[i, j] - b[i, j]) * v[i, t]^2 )
    )
    ## Nodal balance
    @NLconstraint(m, P_bus[i in Buses, t in T],
                    sum(p[g, t] for g in Gens if (g == i)) - DemandData[i, t] - Pch[i, t] + Pdisch[i, t] - 
                    sum(pij[i, j, t] for j in Buses if ((i != j) && (abs(Y[i, j]) > 0.))) == 0
                    
    )

    
    @NLconstraint(m, Q_bus[i in Buses, t in T],
                    sum(q[g, t] for g in Gens if g == i) - BusData[i, :Qd] - 
                    sum(qij[i, j, t] for j in Buses if ((i != j) && (abs(Y[i, j]) > 0.))) == 0
    )
    
    ## Line limits
    @NLconstraint(m, S_max[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                    pij[i, j, t]^2 + qij[i, j, t]^2 ≤ S[i, j]^2
        
    )
    
    ## Phase angle difference
    @constraint(m, δ_diff[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                    -pi ≤ δ[i, t] - δ[j, t] ≤ pi
    )
    
    # Objective
    @objective(m, Min, sum(p[g, t]^2 * GenCostData[g, :c2] + 
                           p[g, t]^1 * GenCostData[g, :c1] + 
                                       GenCostData[g, :c0]
                           for g in Gens for t in T))
    
    # Solve
    JuMP.optimize!(m)
    return m
end

OPF (generic function with 1 method)

In [12]:
# Тут функция с Disistrict OPF, без линеаризации
function DCPF(G, B, b, S, BusData, DemandData, GenData, Gens, Buses, T)
    """
    This function creates a JuMP model for Optimal Power Flow (OPF) problem with provided parameters of power grid
    
    Arguments:
        G (Matrix{Real})        : conductance matrix of grid -- real part of admittance matrix
        B (Matrix{Real})        : susceptance matrix of grid -- imaginary part of admittance matrix
        b (Matrix{Float64})     : shunt matrix for lines
        S (Matrix{Float64})     : line flow limits matrix
        BusData (DataFrame)     : contains information on buses: power demands and voltage magnitude limits
        GenData (DataFrame)     : contains information on generators: generation limits and quadratic cost function's coefficients
        Gens (UnitRange{Int64}) : indices of generators
        Buses (UnitRange{Int64}): indices of buses
    Returns:
        
    """
    
    m = Model(Ipopt.Optimizer)
    Emax = 100.0
    Emin = 10.0
    η_ch = 1.0 #0.8
    η_disch = 0.85
    # Variables
    ## Voltages
    @variable(m, BusData[i, :Vmin] ≤ v[i in Buses, t in T] ≤ BusData[i, :Vmax], start = 1.0)
    @variable(m, δ[i in Buses, t in T], start = 0.0)
    
    ## Power generations
    @variable(m, GenData[i, :Pmin] ≤ p[i in Gens, t in T] ≤ GenData[i, :Pmax])
    @variable(m, GenData[i, :Qmin] ≤ q[i in Gens, t in T] ≤ GenData[i, :Qmax])
        
    # Battery
    @variable(m, Emin ≤ E[t in T] ≤ Emax)
    @variable(m, 0.0 ≤ Pch[i in Buses, t in T] ≤ 25.0)
    @variable(m, 0.0 ≤ Pdisch[i in Buses, t in T] ≤ 40.0)
    
    # No simultaneous charge and discharge
    @constraint(m, Psim[i in Buses, t in T], Pch[i, t] * Pdisch[i, t] == 0.0)
    
    ## Battery energy
    @constraint(m, Et1, E[1] == Emin)
    @constraint(m, Pch2, Pch[2, 1] == 0.0)
    @constraint(m, Pdisch2, Pdisch[2, 1] == 0.0)
    @constraint(m, Pch1[t in T], Pch[1, t] == 0.0)
    @constraint(m, Pdisch1[t in T], Pdisch[1, t] == 0.0) # / η_disch 
    
    @constraint(m, Et[t in 2:24], E[t] == E[t - 1] - (Pdisch[2, t - 1]) +  (η_ch * Pch[2, t - 1]))
    
    @constraint(m, Pdisch21[t in T], E[t] - Pdisch[2, t] ≥ Emin )
    @constraint(m, Pch21[t in T], E[t] + η_ch * Pch[2, t] ≤ Emax )
    
    ## Power flows
    @variable(m, pij[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)])
    @variable(m, qij[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)])
    
    #Constraints
    
    ##Slack bus
    @constraint(m, δ_slack[t in T], δ[1, t] == 0.0)
    
    ## Power flow through lines
    @NLconstraint(m, p_line[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                     pij[i, j, t] == S_base * (δ[i, t] - δ[j, t]) / X[i, j]
    )
    @NLconstraint(m, q_line[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                     qij[i, j, t] == S_base * (v[i, t] - v[j, t])/ X[i, j]
    )
    ## Nodal balance
    @NLconstraint(m, P_bus[i in Buses, t in T],
                    sum(p[g, t] for g in Gens if (g == i)) - DemandData[i, t] - Pch[i, t] + Pdisch[i, t] - 
                    sum(pij[i, j, t] for j in Buses if ((i != j) && (abs(Y[i, j]) > 0.))) == 0
                    
    )

    
    @NLconstraint(m, Q_bus[i in Buses, t in T],
                    sum(q[g, t] for g in Gens if g == i) - BusData[i, :Qd] - 
                    sum(qij[i, j, t] for j in Buses if ((i != j) && (abs(Y[i, j]) > 0.))) == 0
    )
    
    ## Line limits
    @NLconstraint(m, S_max[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                    pij[i, j, t]^2 + qij[i, j, t]^2 ≤ S[i, j]^2
        
    )
    
    ## Phase angle difference
    @constraint(m, δ_diff[i in Buses, j in Buses, t in T; (i != j) && (abs(Y[i, j]) > 0.)],
                    -pi ≤ δ[i, t] - δ[j, t] ≤ pi
    )
    
    # Objective
    @objective(m, Min, sum(p[g, t]^2 * GenCostData[g, :c2] + 
                           p[g, t]^1 * GenCostData[g, :c1] + 
                                       GenCostData[g, :c0]
                           for g in Gens for t in T))
    
    # Solve
    JuMP.optimize!(m)
    return m
end

DCPF (generic function with 1 method)

In [13]:
# @time mOPF = OPF(G, B, b, S, BusData, DemandData, GenData, Gens, Buses, T)

In [14]:
@time mDCPF = DCPF(G, B, b, S, BusData, DemandData, GenData, Gens, Buses, T)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      839
Number of nonzeros in inequality constraint Jacobian.:      384
Number of nonzeros in Lagrangian Hessian.............:      144

Total number of variables............................:      408
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      264
                     variables with only upper bounds:        0
Total number of equal

4538.846273 seconds (38.81 M allocations: 2.286 GiB, 0.09% gc time, 97.82% compilation time)


A JuMP Model
Minimization problem with:
Variables: 408
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 98 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 24 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 24 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 48 constraints
`QuadExpr`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 264 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 264 constraints
Nonlinear: 240 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Ipopt
Names registered in the model: E, Et, Et1, P_bus, Pch, Pch1, Pch2, Pch21, Pdisch, Pdisch1, Pdisch2, Pdisch21, Psim, Q_bus, S_max, p, p_line, pij, q, q_line, qij, v, δ, δ_diff, δ_slack

In [15]:
v = value.(mOPF[Symbol("δ")])

LoadError: UndefVarError: mOPF not defined

In [16]:
p = value.(mOPF[Symbol("p")])
E = value.(mOPF[Symbol("E")])
pdis = value.(mOPF[Symbol("Pdisch")])
pch = value.(mOPF[Symbol("Pch")])

results = DataFrame(P2 = Float64[], P1 = Float64[], E = Float64[], 
                    Pch = Float64[], Pdis = Float64[], Demand = Float64[])

for i in 1:24
    push!(results, round.(tuple(p[2, i], p[1, i], E[i], pch[2, i], pdis[2, i], DemandData[!,i][2]), digits=2))
end

LoadError: UndefVarError: mOPF not defined

In [17]:
results

LoadError: UndefVarError: results not defined

In [18]:
@show DemandData

DemandData = 2×24 DataFrame
 Row │ 1        2        3        4        5        6        7        8        9        10       11       12       13       14       15       16       17       18       19       20       21       22       23       24
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │  0.0       0.0      0.0      0.0      0.0      0.0      0.0       0.0     0.0      0.0     0.0      0.0       0.0     0.0       0.0      0.0     0.0       0.0      0.0       0.0    0.0       0.0     0.0        0.0
   2 │ 20.2195   19.075   18.312   17.549   17.549   17.549   20.601    22.89   26.235   

,1,2,3,4,5,6,7,8,9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.2195,19.075,18.312,17.549,17.549,17.549,20.601,22.89,26.235
